# Определение общих каротажей

In [1]:
import os
import re

import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

Определение пути к директории с каротажами в формате `.npy`, полученными на предыдущем этапе:

In [2]:
WELL_NPY_PATH = "path/to/npy"

Определение пути к выходному файлу со списком общих каротажей:

In [3]:
COMMON_MNEMONICS_PATH = "path/to/common/mnemonics"

Определение пути к выходному файлу со списком скважин, для каждой из которых были сняты все каротажи из выбранного списка:

In [4]:
WELL_LIST_PATH = "path/to/wells"

Определение множества каротажей, измеренных для всех скважин месторождения:

In [5]:
well_names = [file.split("_")[0] for file in os.listdir(WELL_NPY_PATH)
              if re.fullmatch("\d+_meta\.npy", file) is not None]

In [6]:
mnemonics_dict = {}
for well_name in well_names:
    mnemonics = set(np.load(os.path.join(WELL_NPY_PATH, well_name + "_meta.npy")))
    mnemonics_dict[well_name] = mnemonics

Такой каротаж только один — каротаж потенциала собственной поляризации:

In [7]:
common_mnemonics = set.intersection(*mnemonics_dict.values()) - {"DEPT"}
common_mnemonics

{'SP'}

Определим максимальное по мощности множество каротажей, одновременно измеренных как минимум для 90% скважин:

In [8]:
MIN_SUPPORT = 0.9

te = TransactionEncoder()
mnemonics_te = te.fit_transform(mnemonics_dict.values())
mnemonics_df = pd.DataFrame(mnemonics_te, columns=te.columns_)
apriori_df = apriori(mnemonics_df, min_support=MIN_SUPPORT, use_colnames=True)
apriori_df = apriori_df.sort_values(by=["support"], ascending=False)

In [9]:
common_mnemonics = max(apriori_df["itemsets"], key=lambda x: (len(x), "".join(sorted(x)))) - {"DEPT"}
well_names = [name for name in sorted(mnemonics_dict.keys()) if common_mnemonics < mnemonics_dict[name]]

well_names = np.array(well_names)
np.save(WELL_LIST_PATH, well_names)

common_mnemonics = np.array(sorted(common_mnemonics))
np.save(COMMON_MNEMONICS_PATH, common_mnemonics)

В него входят 12 каротажей:

In [10]:
common_mnemonics

array(['CFTC', 'CILD', 'GR', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ7', 'LLD',
       'NKTD', 'PROX', 'SP'], dtype='<U4')

Эти каротажи измерены для 321 скважины:

In [11]:
len(well_names)

321